<a href="https://colab.research.google.com/github/xozi/powerpkg/blob/main/run_powerflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Powerpkg Notebook
The following notebook compiles and runs the latest version of powerpkg with a dropdown for each assignment or examples. It "should" autoconfigure to the runtime GPU you're using.

Please set the GPU by clicking Runtime -> Change Runtime type, and ensure it's a GPU.

In [ ]:
import subprocess
import json
import ipywidgets as widgets
from IPython.display import display

# Install necessary packages
!apt-get update

# Remove the powerpkg directory if it exists and clone the repository
%cd /content
!rm -rf ./powerpkg && git clone https://github.com/xozi/powerpkg.git

# Change to the powerpkg directory
%cd ./powerpkg

def get_gpu_info():
    try:
        # Get nvcc path
        nvcc_path = subprocess.check_output('which nvcc', shell=True).decode().strip()
        print(f"Found nvcc at: {nvcc_path}")

        # Get GPU info
        gpu_info = subprocess.check_output(
            'nvidia-smi --query-gpu=gpu_name --format=csv,noheader',
            shell=True
        ).decode().strip()
        print(f"Found GPU: {gpu_info}")

        return nvcc_path, gpu_info

    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e}")
        return None, None
    except Exception as e:
        print(f"Error: {e}")
        return None, None

# Initialize GPU environment
print("Initializing GPU environment...")
nvcc_path, gpu_info = get_gpu_info()

if nvcc_path and gpu_info:
    print(f"NVCC path: {nvcc_path}")
    print(f"GPU: {gpu_info}")

    # Map GPU model to compute capability
    compute_capability = {
        "Tesla T4": "75",
        "Tesla K80": "37",
        "Tesla P100": "60",
        "Tesla V100": "70",
    }
    arch = compute_capability.get(gpu_info, "75")
    
    !cmake .. -DCMAKE_BUILD_TYPE=Release \
             -DCMAKE_CUDA_COMPILER={nvcc_path} \
             -DCMAKE_EXPORT_COMPILE_COMMANDS=ON \
             -DCMAKE_CXX_COMPILER=/usr/bin/g++ \
             -DCMAKE_CUDA_ARCHITECTURES={arch} \
             -DCMAKE_CUDA_HOST_COMPILER=/usr/bin/gcc \
             -DCMAKE_CUDA_FLAGS="--allow-unsupported-compiler"
    !cmake --build .
else:
    print("Failed to initialize GPU environment")
    exit(1)

def run_test(test_type):
    """Run the powerpkg binary with specified test type"""
    !./powerpkg {test_type}

    # Display results
    print("\nResults:")
    !cat result.txt

# Create dropdown for test selection
test_selector = widgets.Dropdown(
    options=[('Example 1', 'ex1'), ('Homework 1', 'hw1'), ('Example 2', 'ex2'), ('Homework 2', 'hw2')],
    value='ex1',
    description='Test:',
    style={'description_width': 'initial'}
)

# Create run button
run_button = widgets.Button(
    description='Run Test',
    button_style='success',
    tooltip='Click to run the selected test'
)

# Create output area
output = widgets.Output()

# Button click handler
def on_button_click(b):
    with output:
        output.clear_output()
        print(f"Running test: {test_selector.value}")
        run_test(test_selector.value)

run_button.on_click(on_button_click)

# Display widgets
display(widgets.VBox([
    widgets.HBox([test_selector, run_button]),
    output
]))

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
/content
Cloning into 'powerpkg'...
remote: Enumerating objects: 94, d